In [2]:
install.packages("caret")
install.packages("mvtnorm")
install.packages("knockoff")
library(MASS)  
library(glmnet)
library(caret)
library(knockoff) 
library(mvtnorm)


The downloaded binary packages are in
	/var/folders/zf/z_5ht60d3n7gfsmz14cjq6fc0000gn/T//RtmpoOYSOX/downloaded_packages

The downloaded binary packages are in
	/var/folders/zf/z_5ht60d3n7gfsmz14cjq6fc0000gn/T//RtmpoOYSOX/downloaded_packages

The downloaded binary packages are in
	/var/folders/zf/z_5ht60d3n7gfsmz14cjq6fc0000gn/T//RtmpoOYSOX/downloaded_packages


In [3]:
# p1
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from scipy.stats import norm

n, p = 1000, 1000
p0 = 950
sigma = 1
lambda_ = 2
alpha = 0.5
beta1 = 3

np.random.seed(0)
X = np.random.randn(n, p)
beta = np.zeros(p)
beta[0] = beta1
y = X @ beta + np.random.normal(0, sigma, size=n)

model = ElasticNet(alpha=lambda_ * alpha, l1_ratio=alpha, fit_intercept=False)
model.fit(X, y)
beta_hat = model.coef_

beta1_hat = beta_hat[0]
se = np.sqrt(mean_squared_error(y, X @ beta_hat) / n)
z_score = beta1_hat / se
p_value = 2 * (1 - norm.cdf(np.abs(z_score)))
power = 1 - (p_value > 0.05)

print(f"beta_hat:{beta1_hat}")
print(f"power of detection for beta1: {power:.2f}")


ERROR: Error in parse(text = x, srcfile = src): <text>:2:8: unexpected symbol
1: # p1
2: import numpy
          ^


In [ ]:
#p6
W <- c(8, -4, -2, 2, -1.2, -0.6, 10, 12, 1, 5, 6, 7)
q <- 0.4

ratios <- list()
t_values <- sort(unique(abs(W[W > 0])))

for (t in t_values) {
  num_le_neg_t <- sum(W <= -t)
  num_ge_t <- sum(W >= t)
  ratio <- (1 + num_le_neg_t) / num_ge_t
  ratios[[length(ratios) + 1]] <- list(t = t, ratio = ratio)
}

t_hat <- NULL
for (item in ratios) {
  if (item$ratio <= q) {
    t_hat <- item$t
    break
  }
}

important_vars <- if (!is.null(t_hat)) {
  which(W >= t_hat)
} else {
  integer(0)
}
print(important_vars)
# list(t_values, ratios, t_hat, important_vars)
important_values <- W[important_vars]
print(important_values)

[1]  1  7  8 10 11 12
[1]  8 10 12  5  6  7


### computer project

In [19]:
generate_data <- function(n, p) {
    random_matrix <- matrix(rnorm(n * p, sd = sqrt(1/n)), nrow = n)
    X <- qr.Q(qr(random_matrix, LAPACK=TRUE))
    # X <- qr.Q(random_matrix)
    return(X)
}

get_FDP_and_TPP <- function(beta_hat, beta_true)
{
  FP=sum(beta_hat & (beta_true == 0))
  FN=sum(!beta_hat & (beta_true > 0))
  TP=sum(beta_hat & (beta_true > 0))
  Reject=sum(beta_hat != 0)
  FDP <- FP/(max(Reject, 1))
  TPP <- TP/ sum(beta_true > 0)
  returnValue(c(FDP = FDP, TPP=TPP))
}

In [20]:
experiment <- function(X, k_arg) {
  n_arg <- nrow(X)
  p_arg <- ncol(X)
  Beta_true <- c(rep(10, k_arg), rep(0, p_arg - k_arg))
  y <- X %*% Beta_true + rnorm(n_arg, mean = 0, sd = 2)
  
  # ols
  model_OLS <- lm(y ~ X - 1)
  Beta_OLS <- coef(model_OLS)

  # ridge
  lambda_ridge <- cv.glmnet(X, y, alpha = 0, standardize = FALSE)
  Beta_ridge <- coef(lambda_ridge, s = "lambda.1se")[-1]
  
  # Lasso
  lambda_lasso <- cv.glmnet(X, y, alpha = 1, standardize = FALSE)
  Beta_lasso <- coef(lambda_lasso, s = "lambda.1se")[-1]
  
  # Knockoffs for Lasso
  cov_matrix <- diag(1 / n_arg, p_arg)
  knockOff <- function(X) create.gaussian(X, mu = rep(0, length(Beta_true)), Sigma = cov_matrix)
  knockoff_lasso <- knockoff.filter(X, y, knockoffs = knockOff, statistic = stat.glmnet_coefdiff, fdr = 0.2)
  selected_by_knockoff <- rep(0, length(Beta_true))
  selected_by_knockoff[knockoff_lasso$selected] <- TRUE
  discoveries_knockoff_lasso <- get_FDP_and_TPP(beta_hat = selected_by_knockoff, beta_true = Beta_true)
  
  # Knockoffs for Ridge
  k_stat_ridge <- function(X, Xk, y) stat.glmnet_coefdiff(X, Xk, y, alpha = 0)
  knockoff_ridge <- knockoff.filter(X, y, knockoffs = knockOff, statistic = k_stat_ridge, fdr = 0.2)
  selected_by_knockoff <- rep(0, length(Beta_true))
  selected_by_knockoff[knockoff_ridge$selected] <- TRUE
  discoveries_knockoff_ridge <- get_FDP_and_TPP(beta_hat = selected_by_knockoff, beta_true = Beta_true)
  
  # mse
  MSE_OLS <- sum((Beta_true - Beta_OLS)^2)
  MSE_ridge <- sum((Beta_true - Beta_ridge)^2)
  MSE_lasso <- sum((Beta_true - Beta_lasso)^2)
  
  XB <- X %*% Beta_true
  MSE_mean_OLS <- sum((XB - X %*% Beta_OLS)^2)
  MSE_mean_ridge <- sum((XB - X %*% Beta_ridge)^2)
  MSE_mean_lasso <- sum((XB - X %*% Beta_lasso)^2)
  
  discoveries_Lasso <- get_FDP_and_TPP(beta_hat = Beta_lasso, beta_true = Beta_true)

  results_row <- c(MSE_OLS, MSE_ridge, MSE_lasso, MSE_mean_OLS, MSE_mean_ridge, MSE_mean_lasso,
                   discoveries_Lasso, discoveries_knockoff_lasso, discoveries_knockoff_ridge)
  
  return(results_row)
}


In [21]:
n <- 500
p <- 450
k <- c(5, 20, 50)
n_rep <- 1

In [22]:
results <- list()
for (k_value in k) {
  results[[as.character(k_value)]] <- matrix(0, nrow = n_rep, ncol = 12)
  colnames(results[[as.character(k_value)]]) <- c("MSE_OLS", "MSE_Ridge", "MSE_Lasso", 
                                                  "MSE_mean_OLS", "MSE_mean_Ridge", "MSE_mean_Lasso", 
                                                  "FDP_Lasso", "TPP_Lasso", "FDP_Knockoff_Lasso", "TPP_Knockoff_Lasso", 
                                                  "FDP_Knockoff_Ridge", "TPP_Knockoff_Ridge")
}

In [23]:
for (k_value in k) {
  for (i in 1:n_rep) {
    X <- generate_data(n, p)
    run_results <- experiment(X, k_value)
    results[[as.character(k_value)]][i, ] <- run_results
  }
}

In [24]:
summary_results <- list()
for (k_value in k) {
  summary_results[[as.character(k_value)]] <- apply(results[[as.character(k_value)]], 2, mean)
}
print(summary_results)


$`5`
           MSE_OLS          MSE_Ridge          MSE_Lasso       MSE_mean_OLS 
      1900.6561366        500.0000000        240.8040039       1900.6561366 
    MSE_mean_Ridge     MSE_mean_Lasso          FDP_Lasso          TPP_Lasso 
       500.0000000        240.8040039          0.0000000          1.0000000 
FDP_Knockoff_Lasso TPP_Knockoff_Lasso FDP_Knockoff_Ridge TPP_Knockoff_Ridge 
         0.5000000          1.0000000          0.9180328          1.0000000 

$`20`
           MSE_OLS          MSE_Ridge          MSE_Lasso       MSE_mean_OLS 
      2018.0867038       2000.0000000        592.0455193       2018.0867038 
    MSE_mean_Ridge     MSE_mean_Lasso          FDP_Lasso          TPP_Lasso 
      2000.0000000        592.0455193          0.1666667          1.0000000 
FDP_Knockoff_Lasso TPP_Knockoff_Lasso FDP_Knockoff_Ridge TPP_Knockoff_Ridge 
         0.2000000          1.0000000          0.3548387          1.0000000 

$`50`
           MSE_OLS          MSE_Ridge          MSE_Lasso 